In [1]:
#https://www.programmersought.com/article/8007145472/
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir('/content/drive/MyDrive/Machine_Learning/Crack_Detection/Yolo_v4') #/
!pwd
%ls

/content/drive/MyDrive/Machine_Learning/Crack_Detection/Yolo_v4
backup/                           darknet/         predictions_OK2.jpg
bad.list                          data/            predictions_OK.jpg
chart_custom-yolov4-detector.png  Data.ipynb       Yolov4_custom_data.ipynb
chart.png                         predictions.jpg


In [3]:
from os import listdir
from os.path import isfile, join
import argparse
#import cv2
import numpy as np
import sys
import os
import shutil
import random 
import math
 
def IOU(x,centroids):
    '''
    :param x: a ground truth w,h
         :param centroids: collection of anchor w,h[(w,h),(),...], total k
         :return: a set of IoU values ​​for a single ground truth box and all k anchor boxes
    '''
    IoUs = []
    w, h = x # ground truth's w, h
    for centroid in centroids:
          c_w,c_h = centroid #anchorw,h
          if c_w>=w and c_h>=h: #anchor surrounded by ground truth
                iou = w*h/(c_w*c_h)
          elif c_w>=w and c_h<=h: #anchor
                iou = w*c_h/(w*h + (c_w-w)*c_h)
          elif c_w<=w and c_h>=h: #anchor elongated
                iou = c_w*h/(w*h + c_w*(c_h-h))
          else: #ground truth surrounded by anchor means both w,h are bigger than c_w and c_h respectively
                iou = (c_w*c_h)/(w*h)
          IoUs.append(iou) # will become (k,) shape
    return np.array(IoUs)
 
def avg_IOU(X,centroids):
    '''
         :param X: The set of w,h of ground truth[(w,h),(),...]
         :param centroids: collection of anchor w,h[(w,h),(),...], total k
    '''
    n,d = X.shape
    sum = 0.
    for i in range(X.shape[0]):
         sum+= max(IOU(X[i],centroids)) #return a ground truth with the maximum value of all anchors in IoU
    return sum/n # average all ground truth
 
def write_anchors_to_file(centroids,X,anchor_file,input_shape,yolo_version):
    '''
         :param centroids: collection of anchor w,h[(w,h),(),...], total k
         :param X: The set of w,h of ground truth[(w,h),(),...]
         :param anchor_file: anchor and average IoU output path
    '''
    f = open(anchor_file,'w')
    
    anchors = centroids.copy()
    print(anchors.shape)
 
    if yolo_version=='yolov2':
        for i in range(anchors.shape[0]):
                         #yolo has a zoom factor of 32 times for the image, so divide it by 32 here.
                         # If the network architecture changes, according to the actual zoom factor
                         # Find the actual size of the feature map after the anchor is 32 times larger than the zoom (yolov2)
            anchors[i][0]*=input_shape/32.
            anchors[i][1]*=input_shape/32.
    elif yolo_version=='yolov3' or 'yolov4':
        for i in range(anchors.shape[0]):
            # Find the actual size of yolov3 relative to the original image
            anchors[i][0]*=input_shape
            anchors[i][1]*=input_shape
    else:
        print("the yolo version is not right!")
        exit(-1)
 
    widths = anchors[:,0]
    sorted_indices = np.argsort(widths)
 
    print('Anchors = ', anchors[sorted_indices])
        
    for i in sorted_indices[:-1]:
        f.write('%0.2f,%0.2f, '%(anchors[i,0],anchors[i,1]))
 
    #there should not be comma after last anchor, that's why
    f.write('%0.2f,%0.2f\n'%(anchors[sorted_indices[-1:],0],anchors[sorted_indices[-1:],1]))
    
    f.write('%f\n'%(avg_IOU(X,centroids)))
    print()
 
def kmeans(X,centroids,anchor_file,input_shape,yolo_version):
    
    N = X.shape[0] #ground truth
    iterations = 0
    print("centroids",centroids)
    k, dim = centroids.shape #anchor number k and w, h two-dimensional, dim default equal to 2
    Prev_assignments = np.ones(N)*(-1) #Assign the initial label to each ground truth
    iter = 0
    old_D = np.zeros((N,k)) #Initialize each ground truth for each anchor's IoU
    D = np.zeros((N,k))
    while True:
        iter+=1           
        for i in range(N):
            d = 1 - IOU(X[i],centroids)
            D[i,:] = d #D.append(d)
  
        print("iter {}: dists = {}".format(iter,np.sum(np.abs(old_D-D)))) # Calculate the value of each iteration and the previous IoU change
  
        #assign samples to centroids 
        Assignments = np.argmin(D, axis=1) #Assign each ground truth to the anchor number with the smallest distance d

        if (Assignments == Prev_assignments).all() : #If the result of the previous assignment is the same as the result of this time, the anchor and the average IoU are output.
              print("Centroids = ",centroids)
              write_anchors_to_file(centroids,X,anchor_file,input_shape,yolo_version)
              return False
 
        #calculate new centroids
        Centroid_sums=np.zeros((k,dim),np.float) #initialize to sum w and h for each cluster
        for i in range(N):
              Centroid_sums[Assignments[i]]+=X[i] # accumulate the ground truths w and h in each cluster

        for j in range(k): #averaging w,h in the cluster
            if np.sum(Assignments==j) >0:
                centroids[j] = Centroid_sums[j]/(np.sum(Assignments==j))
            else: centroids[j] = Centroid_sums[j]/(np.sum(Assignments==j)+1)
        
        Prev_assignments = Assignments.copy()     
        old_D = D.copy()  

In [4]:
def main(argv):
    parser = argparse.ArgumentParser()
    parser.add_argument('-filelist', default = r'data/train/train.txt',
                        help='path to filelist\n' )
    parser.add_argument('-output_dir', default = r'data/train/anchors', type = str,
                        help='Output anchor directory\n' )
    parser.add_argument('-num_clusters', default = 9, type = int, 
                        help='number of clusters\n' )
    parser.add_argument('-yolo_version', default='yolov4', type=str,
                        help='yolov2 or yolov3\n')
    parser.add_argument('-yolo_input_shape', default=416, type=int,
                        help='input images shape，multiples of 32. etc. 416*416\n')
    args = parser.parse_args()
    
    if not os.path.exists(args.output_dir):
        os.mkdir(args.output_dir)
 
    #f = open(args.filelist, 'r')
    f = open('data/train/train.txt', 'r')
  
    lines = [line.rstrip('\n') for line in f.readlines()]
    
    annotation_dims = []
 
    for line in lines:
        line = line.replace('.jpg','.txt')
        print(line)
        check = line[11:14]
        if check == 'Pos':
            f2 = open(line, 'r')
            for line in f2.readlines():
                line = line.rstrip('\n')
                w,h = line.split(' ')[2:4]            
                annotation_dims.append((float(w),float(h)))
                Annotation_dims = np.array(annotation_dims) #Save all ground truth boxes (w, h)

    for num_clusters in range(1,10): #we make 1 through 9 clusters 
        anchor_file = join( args.output_dir,'anchors%d.txt'%(num_clusters))
        indices = [random.randrange(Annotation_dims.shape[0]) for i in range(num_clusters)]
        centroids = Annotation_dims[indices]
        kmeans(Annotation_dims,centroids,anchor_file,args.yolo_input_shape,args.yolo_version)
        print('centroids.shape', centroids.shape)
 
if __name__=="__main__":
    main(sys.argv)

data/train/Neg_00050.txt
data/train/Neg_01070.txt
data/train/Neg_01050.txt
data/train/Neg_00020.txt
data/train/Pos_00070.txt
data/train/Neg_01100.txt
data/train/Neg_01150.txt
data/train/Pos_00072.txt
data/train/Neg_00540.txt
data/train/Pos_00075.txt
data/train/Neg_00040.txt
data/train/Neg_00560.txt
data/train/Neg_01130.txt
data/train/Neg_00600.txt
data/train/Pos_00078.txt
data/train/Pos_00069.txt
data/train/Neg_00580.txt
data/train/Neg_00500.txt
data/train/Pos_00158.txt
data/train/Pos_00151.txt
data/train/Pos_00161.txt
data/train/Pos_00093.txt
data/train/Pos_00153.txt
data/train/Pos_00088.txt
data/train/Pos_00160.txt
data/train/Pos_00094.txt
data/train/Pos_00169.txt
data/train/Pos_00085.txt
data/train/Pos_00097.txt
data/train/Pos_00096.txt
data/train/Pos_00089.txt
data/train/Pos_00099.txt
data/train/Pos_00082.txt
data/train/Pos_00164.txt
data/train/Pos_00090.txt
data/train/Pos_00100.txt
data/train/Pos_00150.txt
data/train/Pos_00155.txt
data/train/Pos_00086.txt
data/train/Pos_00095.txt
